<a href="https://colab.research.google.com/github/irmakguney/WealthGauge/blob/main/Portfolio_suggestions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP Setup & Data Fetching

**1️⃣ Set up Colab notebook & packages and portfolio input**

In [3]:
import pandas as pd
import yfinance as yf
import requests
import json

# Step 1: Ask user for portfolio
portfolio = {}
print("Enter your portfolio (type 'done' when finished):")
while True:
    asset_type = input("Is this a Stock or Crypto? (stock/crypto/done): ").lower()
    if asset_type == 'done':
        break
    symbol = input("Enter ticker/symbol (e.g., AAPL or ETH): ").upper()
    amount = float(input(f"How much of {symbol} do you have? "))
    portfolio[symbol] = {"type": asset_type, "amount": amount}

# Step 2: Save portfolio to a JSON file
with open("portfolio.json", "w") as f:
    json.dump(portfolio, f)

print("\n✅ Portfolio saved successfully!")
print(portfolio)

# Step 3: Load portfolio from file later
with open("portfolio.json", "r") as f:
    saved_portfolio = json.load(f)

print("\n---- Your Saved Portfolio ----")
for symbol, info in saved_portfolio.items():
    print(f"{symbol} ({info['type']}): {info['amount']}")

Enter your portfolio (type 'done' when finished):
Is this a Stock or Crypto? (stock/crypto/done): avax
Enter ticker/symbol (e.g., AAPL or ETH): avax
How much of AVAX do you have? 2
Is this a Stock or Crypto? (stock/crypto/done): stock
Enter ticker/symbol (e.g., AAPL or ETH): aapl
How much of AAPL do you have? 10
Is this a Stock or Crypto? (stock/crypto/done): crypto
Enter ticker/symbol (e.g., AAPL or ETH): btc
How much of BTC do you have? 10
Is this a Stock or Crypto? (stock/crypto/done): done

✅ Portfolio saved successfully!
{'AVAX': {'type': 'avax', 'amount': 2.0}, 'AAPL': {'type': 'stock', 'amount': 10.0}, 'BTC': {'type': 'crypto', 'amount': 10.0}}

---- Your Saved Portfolio ----
AVAX (avax): 2.0
AAPL (stock): 10.0
BTC (crypto): 10.0


**Fetch Live Prices & Calculate Current Portfolio Value**

In [5]:
import yfinance as yf
import requests
import json

# Mapping crypto tickers to CoinGecko IDs
crypto_ids = {
    "BTC": "bitcoin",
    "ETH": "ethereum",
    "BNB": "binancecoin",
    "DOGE": "dogecoin",
    "SOL": "solana"
}

# Load saved portfolio
with open("portfolio.json", "r") as f:
    saved_portfolio = json.load(f)

print("\n---- Fetching Live Prices ----")
total_value = 0

for symbol, info in saved_portfolio.items():
    if info['type'] == "stock":
        data = yf.Ticker(symbol).history(period="1d")
        current_price = data['Close'].iloc[-1]  # Correct indexing
        value = current_price * info['amount']
        print(f"{symbol} (Stock): ${current_price:.2f} × {info['amount']} = ${value:.2f}")

    elif info['type'] == "crypto":
        cid = crypto_ids.get(symbol.upper())
        if cid:
            url = f"https://api.coingecko.com/api/v3/simple/price?ids={cid}&vs_currencies=usd"
            response = requests.get(url).json()
            if cid in response and "usd" in response[cid]:
                price = response[cid]['usd']
                value = price * info['amount']
                print(f"{symbol} (Crypto): ${price:.2f} × {info['amount']} = ${value:.2f}")
            else:
                print(f"⚠️ Error: Unexpected API response for {symbol}")
                value = 0
        else:
            print(f"⚠️ No CoinGecko ID found for {symbol}")
            value = 0

    total_value += value

print("\n💰 Total Portfolio Value: ${:.2f}".format(total_value))


---- Fetching Live Prices ----
AAPL (Stock): $229.65 × 10.0 = $2296.50
BTC (Crypto): $119897.00 × 10.0 = $1198970.00

💰 Total Portfolio Value: $2400176.50


**Profit/Loss Calculation Code**

In [6]:
print("\n---- Portfolio Performance ----")
total_value = 0
total_invested = 0

for symbol, info in saved_portfolio.items():
    buy_price = float(input(f"Enter your buy price for {symbol}: "))

    if info['type'] == "stock":
        data = yf.Ticker(symbol).history(period="1d")
        current_price = data['Close'].iloc[-1]
    elif info['type'] == "crypto":
        cid = crypto_ids.get(symbol.upper())
        url = f"https://api.coingecko.com/api/v3/simple/price?ids={cid}&vs_currencies=usd"
        response = requests.get(url)
        current_price = response.json().get(cid, {}).get('usd', 0)
    else:
        current_price = 0

    invested = buy_price * info['amount']
    current_value = current_price * info['amount']
    profit_loss = current_value - invested
    change_percent = (profit_loss / invested) * 100 if invested else 0

    total_value += current_value
    total_invested += invested

    status = "📈 Profit" if profit_loss > 0 else "📉 Loss"
    print(f"{symbol} ({info['type']}): ${current_price:.2f} × {info['amount']} = ${current_value:,.2f}")
    print(f"   {status}: ${profit_loss:,.2f} ({change_percent:.2f}%)\n")

print(f"💰 Total Current Value: ${total_value:,.2f}")
print(f"💵 Total Invested: ${total_invested:,.2f}")
print(f"📊 Overall P/L: ${total_value - total_invested:,.2f} ({((total_value-total_invested)/total_invested)*100:.2f}%)")


---- Portfolio Performance ----
Enter your buy price for AVAX: 2
AVAX (avax): $0.00 × 2.0 = $0.00
   📉 Loss: $-4.00 (-100.00%)

Enter your buy price for AAPL: 20
AAPL (stock): $229.65 × 10.0 = $2,296.50
   📈 Profit: $2,096.50 (1048.25%)

Enter your buy price for BTC: 10
BTC (crypto): $119893.00 × 10.0 = $1,198,930.00
   📈 Profit: $1,198,830.00 (1198830.00%)

💰 Total Current Value: $1,201,226.50
💵 Total Invested: $304.00
📊 Overall P/L: $1,200,922.50 (395040.30%)


# Rule-Based “AI” Signals

**Detect Trend + Profit Context → Generate Actions**

In [7]:
# ===== Phase 2 — Step 1: Rule-Based Signals (No ML) =====
import json, requests, math
import yfinance as yf
import pandas as pd
from datetime import datetime

# --- Load saved portfolio (from Phase 1 Step 1) ---
with open("portfolio.json", "r") as f:
    saved_portfolio = json.load(f)

# --- CoinGecko ticker -> id map (extend as needed) ---
crypto_ids = {
    "BTC": "bitcoin",
    "ETH": "ethereum",
    "BNB": "binancecoin",
    "DOGE": "dogecoin",
    "SOL": "solana"
}

# --- Helpers: current price + momentum (7d/30d) ---
def stock_snapshot(ticker: str):
    # Pull 60 days to compute 30d momentum robustly
    df = yf.Ticker(ticker).history(period="60d")
    if df.empty:
        return None
    cur = float(df["Close"].iloc[-1])
    # 7d change: compare last close vs close 7 trading days ago (fallbacks)
    idx_7 = -min(len(df), 7)
    idx_30 = -min(len(df), 30)
    c7 = float(df["Close"].iloc[idx_7]) if len(df) >= 7 else float(df["Close"].iloc[0])
    c30 = float(df["Close"].iloc[idx_30]) if len(df) >= 30 else float(df["Close"].iloc[0])
    chg_7d = (cur - c7) / c7 if c7 else 0.0
    chg_30d = (cur - c30) / c30 if c30 else 0.0
    # 1d change (if available)
    if len(df) >= 2:
        chg_1d = (df["Close"].iloc[-1] - df["Close"].iloc[-2]) / df["Close"].iloc[-2]
    else:
        chg_1d = 0.0
    return {"price": cur, "chg_1d": chg_1d, "chg_7d": chg_7d, "chg_30d": chg_30d}

def coin_gecko_history(cid: str, days: int = 30):
    # Returns daily closes for 'days' days
    url = f"https://api.coingecko.com/api/v3/coins/{cid}/market_chart?vs_currency=usd&days={days}&interval=daily"
    r = requests.get(url, timeout=20)
    r.raise_for_status()
    data = r.json()
    # prices = [[timestamp_ms, price], ...]
    closes = [p[1] for p in data.get("prices", [])]
    return closes

def crypto_snapshot(symbol: str):
    cid = crypto_ids.get(symbol.upper())
    if not cid:
        return None
    # Get up to 30 days of daily prices
    closes = coin_gecko_history(cid, days=30)
    if not closes:
        return None
    cur = float(closes[-1])
    c7 = float(closes[-7]) if len(closes) >= 7 else float(closes[0])
    c30 = float(closes[-30]) if len(closes) >= 30 else float(closes[0])
    chg_7d = (cur - c7) / c7 if c7 else 0.0
    chg_30d = (cur - c30) / c30 if c30 else 0.0

    # 1d change (last vs prev)
    chg_1d = 0.0
    if len(closes) >= 2 and closes[-2] != 0:
        chg_1d = (closes[-1] - closes[-2]) / closes[-2]

    return {"price": cur, "chg_1d": chg_1d, "chg_7d": chg_7d, "chg_30d": chg_30d}

# --- Rule engine: simple action logic ---
def action_from_metrics(entry_price, current_price, chg_1d, chg_7d, chg_30d):
    if not entry_price or entry_price <= 0:
        entry_price = current_price  # avoid div by zero

    pnl_pct = (current_price - entry_price) / entry_price

    # Define “hype” by short-term momentum
    strong_up = (chg_7d >= 0.08) or (chg_30d >= 0.15)
    mild_up   = (chg_7d >= 0.03) or (chg_30d >= 0.07)
    flat_ish  = abs(chg_7d) < 0.02 and abs(chg_30d) < 0.04
    dumpy     = (chg_7d <= -0.06) or (chg_30d <= -0.12)

    # Fake breakout risk: big 1d pop but weak 7–30d context
    fake_breakout_risk = (chg_1d >= 0.04) and (chg_7d < 0.03 and chg_30d < 0.06)

    # Core scenarios
    if pnl_pct >= 0.5 and strong_up:
        action = "💰 Consider partial take-profit (lock gains) — trend still strong; trail the rest."
        reason = "Up big from entry and momentum is hot."
    elif pnl_pct >= 0.2 and (strong_up or mild_up):
        action = "✅ Keep riding, maybe set a stop or trim a little."
        reason = "Healthy profit and positive short-term momentum."
    elif pnl_pct > 0 and fake_breakout_risk:
        action = "⚠️ Hold but watch for fake breakout — avoid chasing; wait for confirmation."
        reason = "Sharp daily pop without broader support."
    elif pnl_pct > 0 and flat_ish:
        action = "🙂 Mild profit; could hold or trim small — momentum is meh."
        reason = "Sideways vibe despite gains."
    elif pnl_pct <= -0.15 and dumpy:
        action = "🧯 Cut risk / reassess thesis — downtrend + sizable drawdown."
        reason = "Momentum is negative and losses are stacking."
    elif pnl_pct < 0 and not dumpy:
        action = "😬 Underwater but trend not awful — set invalidation level; avoid revenge buys."
        reason = "Loss present but momentum mixed."
    else:
        action = "🤔 Neutral — wait for clearer momentum or levels."
        reason = "No strong signal."

    return pnl_pct, action, reason

# --- Run signals for each asset (asks for your entry per asset once) ---
signals = []
print("\n===== Phase 2 — Step 1: Signals =====")
for symbol, info in saved_portfolio.items():
    try:
        entry = float(input(f"Enter your average buy price for {symbol}: ").strip())
    except Exception:
        entry = 0.0

    snap = None
    if info["type"] == "stock":
        snap = stock_snapshot(symbol)
    elif info["type"] == "crypto":
        snap = crypto_snapshot(symbol)

    if not snap:
        print(f"⚠️ Skipping {symbol}: couldn’t fetch snapshot.")
        continue

    pnl_pct, action, reason = action_from_metrics(
        entry_price=entry,
        current_price=snap["price"],
        chg_1d=snap["chg_1d"],
        chg_7d=snap["chg_7d"],
        chg_30d=snap["chg_30d"],
    )

    signals.append({
        "symbol": symbol,
        "type": info["type"],
        "entry": entry,
        "price": snap["price"],
        "pnl_pct": pnl_pct,
        "chg_1d": snap["chg_1d"],
        "chg_7d": snap["chg_7d"],
        "chg_30d": snap["chg_30d"],
        "action": action,
        "reason": reason
    })

# --- Pretty print table ---
df = pd.DataFrame(signals)
if not df.empty:
    df["pnl_pct%"]  = (df["pnl_pct"]  * 100).round(2)
    df["chg_1d%"]   = (df["chg_1d"]   * 100).round(2)
    df["chg_7d%"]   = (df["chg_7d"]   * 100).round(2)
    df["chg_30d%"]  = (df["chg_30d"]  * 100).round(2)
    view = df[["symbol","type","entry","price","pnl_pct%","chg_1d%","chg_7d%","chg_30d%","action","reason"]]
    print("\n--- Signals ---")
    print(view.to_string(index=False))
else:
    print("No signals to show.")


===== Phase 2 — Step 1: Signals =====
Enter your average buy price for AVAX: 100
⚠️ Skipping AVAX: couldn’t fetch snapshot.
Enter your average buy price for AAPL: 10
Enter your average buy price for BTC: 1000000

--- Signals ---
symbol   type     entry         price  pnl_pct%  chg_1d%  chg_7d%  chg_30d%                                                                            action                                 reason
  AAPL  stock      10.0    229.649994   2196.50     1.09    13.06     10.63 💰 Consider partial take-profit (lock gains) — trend still strong; trail the rest. Up big from entry and momentum is hot.
   BTC crypto 1000000.0 119893.739929    -88.01    -0.26     2.07      1.88    😬 Underwater but trend not awful — set invalidation level; avoid revenge buys.       Loss present but momentum mixed.


**Risk & Diversification Rules + Breakout/Take-Profit Tiers**

In [8]:
# ===== Phase 2 — Step 2: Risk & Diversification Rules + Breakout/Take-Profit Tiers =====
import json, requests, math
import pandas as pd
import numpy as np
import yfinance as yf

# --- Ensure we have portfolio + crypto_ids from previous steps ---
with open("portfolio.json", "r") as f:
    saved_portfolio = json.load(f)

crypto_ids = {
    "BTC": "bitcoin",
    "ETH": "ethereum",
    "BNB": "binancecoin",
    "DOGE": "dogecoin",
    "SOL": "solana"
}

# Try to reuse entries from Phase 2 Step 1 if present; else ask again
entries = {}
try:
    # If a 'signals' list or df exists from Step 1, pull entries from it
    _df_try = pd.DataFrame(signals)  # may raise if not defined
    for r in signals:
        entries[r["symbol"]] = r.get("entry", 0.0)
except Exception:
    # Ask for avg buy price per symbol
    print("Enter your average buy price per asset (from Step 1 if you remember):")
    for sym in saved_portfolio:
        try:
            entries[sym] = float(input(f"- {sym} entry: ").strip())
        except Exception:
            entries[sym] = 0.0

# --- Helpers ---
def cg_history(cid: str, days: int = 60):
    url = f"https://api.coingecko.com/api/v3/coins/{cid}/market_chart?vs_currency=usd&days={days}&interval=daily"
    r = requests.get(url, timeout=20)
    r.raise_for_status()
    data = r.json().get("prices", [])
    closes = [p[1] for p in data]
    return pd.Series(closes, dtype="float64")

def stock_history(ticker: str, days: str = "3mo"):
    df = yf.Ticker(ticker).history(period=days)
    if df.empty:
        return pd.Series([], dtype="float64")
    return df["Close"].astype("float64")

def snapshot_series(symbol: str, a_type: str):
    if a_type == "stock":
        s = stock_history(symbol, "3mo")
    else:
        cid = crypto_ids.get(symbol.upper())
        if not cid:
            return pd.Series([], dtype="float64")
        s = cg_history(cid, 60)
    return s.dropna()

def compute_metrics(series: pd.Series):
    if series.empty:
        return None
    cur = float(series.iloc[-1])
    # 20d high/low
    win = series.iloc[-20:] if len(series) >= 20 else series
    high20 = float(win.max())
    low20  = float(win.min())
    # Simple volatility: 14d std of daily returns
    if len(series) >= 15:
        rets = series.pct_change().dropna().iloc[-14:]
        vol14 = float(rets.std())  # ~daily sigma
    else:
        vol14 = 0.0
    # Simple MAs
    sma20 = float(series.rolling(20).mean().iloc[-1]) if len(series) >= 20 else float(series.mean())
    return {"price": cur, "high20": high20, "low20": low20, "vol14": vol14, "sma20": sma20}

# --- Build current valuation, weights ---
current_rows = []
total_value = 0.0
total_crypto_value = 0.0

for sym, info in saved_portfolio.items():
    ser = snapshot_series(sym, info["type"])
    met = compute_metrics(ser)
    if not met:
        continue
    qty = float(info["amount"])
    cur_val = met["price"] * qty
    total_value += cur_val
    if info["type"] == "crypto":
        total_crypto_value += cur_val
    current_rows.append({
        "symbol": sym,
        "type": info["type"],
        "qty": qty,
        "entry": float(entries.get(sym, 0.0)),
        "price": met["price"],
        "value": cur_val,
        **met
    })

if total_value <= 0 or not current_rows:
    raise SystemExit("No data to process. Check portfolio or network.")

df = pd.DataFrame(current_rows)
df["weight"] = df["value"] / total_value
df["pnl_pct"] = (df["price"] - df["entry"]).where(df["entry"]>0, 0.0) / df["entry"].replace(0, np.nan)
df["pnl_pct"] = df["pnl_pct"].fillna(0.0)

# --- Event/condition flags ---
df["breakout_up"]   = df["price"] > (df["high20"] * 1.01)
df["breakdown_down"]= df["price"] < (df["low20"]  * 0.99)
df["high_vol"]      = df["vol14"] > 0.05  # ~5% daily sigma threshold (tweakable)

# --- Rule layer: tiered actions, stops, notes ---
actions = []
stops   = []
notes   = []

for i, r in df.iterrows():
    pnl = r["pnl_pct"]
    breakout = bool(r["breakout_up"])
    breakdown = bool(r["breakdown_down"])
    high_vol = bool(r["high_vol"])
    price = r["price"]
    sma20 = r["sma20"]

    # Trailing/stop hints (NOT financial advice; just mechanics)
    trail_hint = max(price * 0.92, sma20 * 0.97)  # ~8% trailing, or ~3% below 20d MA
    stop_hint  = min(price * 0.90, sma20 * 0.95)  # tighter hard stop idea

    # Action logic (layered)
    if pnl >= 0.5 and breakout:
        act = "💰 Strong winner + breakout — consider trimming 10–25% and trail the rest."
        nt  = "Lock some gains while momentum is hot."
    elif pnl >= 0.2 and breakout:
        act = "✅ Winner holding trend — ride it; optional small trim, trail stops."
        nt  = "Positive momentum with cushion."
    elif pnl > 0.1 and not breakout and not breakdown:
        act = "🙂 Green and steady — keep holding; watch 20d MA as guide."
        nt  = "No major signal; trend okay."
    elif pnl > 0 and breakdown and high_vol:
        act = "⚠️ Green but shaky — watch for bull trap; tighten stops."
        nt  = "Breakdown + high vol = trap risk."
    elif pnl < 0 and breakdown and high_vol:
        act = "🧯 Down + breakdown — consider reducing risk / waiting for base."
        nt  = "Momentum against you."
    elif pnl < 0 and not breakdown:
        act = "😬 Red but not broken — define invalidation; avoid averaging blindly."
        nt  = "Wait for strength."
    else:
        act = "🤔 Neutral — no clean edge; wait for setup."
        nt  = "Patience saves capital."

    actions.append(act)
    stops.append(f"Trail ≈ {trail_hint:.2f}, Stop ≈ {stop_hint:.2f}")
    notes.append(nt)

df["action"] = actions
df["stops"]  = stops
df["note"]   = notes

# --- Portfolio-level risk flags ---
concentration_flags = []
top_weight = df.loc[df["weight"].idxmax(), ["symbol","weight"]]
if top_weight["weight"] >= 0.40:
    concentration_flags.append(f"⚠️ Concentration: {top_weight['symbol']} is {top_weight['weight']*100:.1f}% of portfolio.")

crypto_share = total_crypto_value / total_value
if crypto_share >= 0.60:
    concentration_flags.append(f"⚠️ Crypto-heavy: {crypto_share*100:.1f}% of portfolio in crypto.")

# --- Present results ---
out = df.copy()
out["weight%"]  = (out["weight"] * 100).round(2)
out["pnl%"]     = (out["pnl_pct"] * 100).round(2)
out["vol14%"]   = (out["vol14"] * 100).round(2)
present_cols = ["symbol","type","qty","entry","price","value","weight%","pnl%","vol14%","high20","low20","breakout_up","breakdown_down","action","stops","note"]
print("\n--- Enhanced Signals (Step 2) ---")
print(out[present_cols].to_string(index=False))

if concentration_flags:
    print("\n--- Portfolio Risk Notes ---")
    for f in concentration_flags:
        print(f"- {f}")

print(f"\nTotal portfolio value: ${total_value:,.2f}")


--- Enhanced Signals (Step 2) ---
symbol   type  qty     entry         price        value  weight%    pnl%  vol14%        high20         low20  breakout_up  breakdown_down                                                               action                               stops                         note
  AAPL  stock 10.0      10.0    229.649994 2.296500e+03     0.19 2196.50    2.18    229.649994    202.150589        False           False            🙂 Green and steady — keep holding; watch 20d MA as guide.       Trail ≈ 211.28, Stop ≈ 202.62 No major signal; trend okay.
   BTC crypto 10.0 1000000.0 119928.183416 1.199282e+06    99.81  -88.01    1.37 120202.534855 112554.902322        False           False 😬 Red but not broken — define invalidation; avoid averaging blindly. Trail ≈ 113364.88, Stop ≈ 107935.37           Wait for strength.

--- Portfolio Risk Notes ---
- ⚠️ Concentration: BTC is 99.8% of portfolio.
- ⚠️ Crypto-heavy: 99.8% of portfolio in crypto.

Total portfolio value:

**AI-Style Human Summary of Portfolio Analysis**

In [9]:
# ===== Phase 2 — Step 3: AI-Style Human Report (no re-entry, uses Step 2 data) =====
import pandas as pd
from datetime import datetime

def build_human_report(df, total_value, concentration_flags=None):
    concentration_flags = concentration_flags or []
    dfx = df.copy()

    # Compute invested & P/L using existing entries and qty (entry may be 0 if not provided)
    dfx["invested"] = dfx.apply(lambda r: (r["entry"] * r["qty"]) if r["entry"] > 0 else 0.0, axis=1)
    dfx["pnl_abs"] = dfx["value"] - dfx["invested"]

    total_invested = float(dfx["invested"].sum())
    overall_pl = float(dfx["pnl_abs"].sum())
    overall_pct = (overall_pl / total_invested) if total_invested > 0 else None

    # Top weights snapshot
    top3 = dfx.sort_values("weight", ascending=False).head(3)

    lines = []
    lines.append(f"🧠 Portfolio Brief — {datetime.now().strftime('%Y-%m-%d %H:%M')}")
    lines.append(f"Total value: ${total_value:,.2f}")
    if total_invested > 0:
        lines.append(f"Overall P/L: {overall_pl:+,.2f} ({overall_pct*100:.2f}%)")
    else:
        lines.append("Overall P/L: — (add entry prices to unlock full P/L)")

    # Top weights line
    if not top3.empty:
        tw = " | ".join([f"{r.symbol}: {r.weight*100:.1f}%" for r in top3.itertuples(index=False)])
        lines.append(f"Top weights: {tw}")

    # Portfolio risk notes
    if concentration_flags:
        lines.append("\nRisk notes:")
        for f in concentration_flags:
            lines.append(f" • {f}")

    # Per-asset section (uses columns created in Step 2)
    lines.append("\nPer-asset view:")
    show_cols_missing = [c for c in ["action","note","stops","high20","low20"] if c not in dfx.columns]
    for r in dfx.sort_values("weight", ascending=False).itertuples(index=False):
        pnl_pct_str = "—" if pd.isna(r.pnl_pct) else f"{r.pnl_pct*100:.2f}%"
        base = f"• {r.symbol} ({r.type}) — Px ${r.price:,.2f} | Qty {r.qty:g} | Wt {r.weight*100:.1f}% | P/L {pnl_pct_str}"
        lines.append(base)

        # Action / note / guardrails if present
        if "action" in dfx.columns:
            lines.append(f"   Action: {r.action}")
        if "note" in dfx.columns:
            lines.append(f"   Note: {r.note}")
        if "stops" in dfx.columns:
            lines.append(f"   Guardrails: {r.stops}")
        if "high20" in dfx.columns and "low20" in dfx.columns:
            lines.append(f"   20d Hi/Lo: {getattr(r,'high20',float('nan')):,.2f}/{getattr(r,'low20',float('nan')):,.2f}")

    # Portfolio-level playbook
    if total_invested > 0:
        if overall_pct is not None and overall_pct >= 0.50:
            lines.append("\n🏁 Playbook: Up big overall — trim 10–25% from winners, trail stops on the rest.")
        elif overall_pct is not None and overall_pct >= 0.10:
            lines.append("\n📈 Playbook: Green and trending — ride strength, tighten risk on any breakdowns.")
        elif overall_pct is not None and overall_pct >= -0.10:
            lines.append("\n😐 Playbook: Near flat — wait for clean momentum before adding risk.")
        else:
            lines.append("\n🧯 Playbook: Drawdown — cut weak links, wait for bases to form.")
    else:
        lines.append("\nℹ️ Add/record your entry prices to enable full P/L guidance per asset.")

    # Heads-up if any analysis columns were missing (e.g., if Step 2 wasn’t run)
    if show_cols_missing:
        lines.append(f"\n(Heads-up: missing columns from Step 2: {', '.join(show_cols_missing)}. Run Phase 2 Step 2 first for full context.)")

    return "\n".join(lines)

# Use the df/total_value/concentration_flags produced in Phase 2 Step 2:
print(build_human_report(df, total_value, concentration_flags if 'concentration_flags' in globals() else []))

🧠 Portfolio Brief — 2025-08-13 09:39
Total value: $1,201,578.33
Overall P/L: -8,798,521.67 (-87.98%)
Top weights: BTC: 99.8% | AAPL: 0.2%

Risk notes:
 • ⚠️ Concentration: BTC is 99.8% of portfolio.
 • ⚠️ Crypto-heavy: 99.8% of portfolio in crypto.

Per-asset view:
• BTC (crypto) — Px $119,928.18 | Qty 10 | Wt 99.8% | P/L -88.01%
   Action: 😬 Red but not broken — define invalidation; avoid averaging blindly.
   Note: Wait for strength.
   Guardrails: Trail ≈ 113364.88, Stop ≈ 107935.37
   20d Hi/Lo: 120,202.53/112,554.90
• AAPL (stock) — Px $229.65 | Qty 10 | Wt 0.2% | P/L 2196.50%
   Action: 🙂 Green and steady — keep holding; watch 20d MA as guide.
   Note: No major signal; trend okay.
   Guardrails: Trail ≈ 211.28, Stop ≈ 202.62
   20d Hi/Lo: 229.65/202.15

🧯 Playbook: Drawdown — cut weak links, wait for bases to form.
